**인공지능 실습 3_2 - Resnet 쉽게 구현**

In [2]:
import tensorflow as tf

In [3]:
# dataset 준비
from keras.datasets import cifar10
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 13s 0us/step


In [4]:
# preprocessing
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
# validation set 준비
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True)

In [6]:
# one-hot vector 만들기
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
encoder.fit(Y_train)

Y_train = encoder.transform(Y_train).toarray()
Y_val = encoder.transform(Y_val).toarray()
Y_test = encoder.transform(Y_test).toarray()

In [7]:
# keras에서 제공하는 ResNet (ImageNet 기준)
resnet = tf.keras.applications.resnet50.ResNet50(include_top=False, input_shape=(32, 32, 3)) # top을 우리가 직접 구현하기 위해서 False로
resnet.summary()

94765736/94765736 [==============================] - 5s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                          

In [8]:
# top 부분 연결
top = resnet.output
top = tf.keras.layers.GlobalAveragePooling2D()(top)
top = tf.keras.layers.Flatten()(top)
top = tf.keras.layers.Dense(10, activation='softmax')(top)

In [9]:
# 모델 생성
model = tf.keras.Model(resnet.input, top)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# training
model.fit(X_train, Y_train, batch_size=256, epochs=10, validation_data=(X_val, Y_val))

Epoch 1/10
157/157 [==============================] - 32s 110ms/step - loss: 1.3296 - accuracy: 0.5975 - val_loss: 3.4552 - val_accuracy: 0.0979
Epoch 2/10
157/157 [==============================] - 15s 95ms/step - loss: 0.8972 - accuracy: 0.6964 - val_loss: 4.7173 - val_accuracy: 0.0979
Epoch 3/10
157/157 [==============================] - 15s 96ms/step - loss: 0.5726 - accuracy: 0.8025 - val_loss: 3.8036 - val_accuracy: 0.1545
Epoch 4/10
157/157 [==============================] - 15s 97ms/step - loss: 0.3998 - accuracy: 0.8619 - val_loss: 3.7011 - val_accuracy: 0.1659
Epoch 5/10
157/157 [==============================] - 15s 98ms/step - loss: 0.3022 - accuracy: 0.8959 - val_loss: 2.1569 - val_accuracy: 0.4370
Epoch 6/10
157/157 [==============================] - 15s 98ms/step - loss: 0.2470 - accuracy: 0.9138 - val_loss: 1.7525 - val_accuracy: 0.5801
Epoch 7/10
157/157 [==============================] - 15s 98ms/step - loss: 0.2020 - accuracy: 0.9305 - val_loss: 0.8908 - val_accuracy

In [11]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 4s 13ms/step - loss: 1.0141 - accuracy: 0.7589


[1.0140560865402222, 0.758899986743927]